In [1]:
# make sure that rpy2 is installed: https://rpy2.bitbucket.io/
%load_ext rpy2.ipython

import warnings
from rpy2.rinterface import RRuntimeWarning
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

# ignore R warning messages
warnings.filterwarnings("ignore", category=RRuntimeWarning)
import anndata

/home/wenhao/miniconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
%%R -o N1_DAT  -o geneinfo_N1 -o cellinfo_N1 

load('/home/wenhao/RAW_REAL/RAW_REAL/RAW_SCNORM_H1.RData')
qq<-round(cbind(H1_p24,H1_p96)[whichg_H1,])

cellinfo_N1<-data.frame(Cell=colnames(qq))
geneinfo_N1<-data.frame(Gene=rownames(qq))

N1_DAT<-as.data.frame(t(qq))


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy.api as sc
import DCA.api as dca

In [8]:
adata_N1 = sc.AnnData(N1_DAT.values,var=geneinfo_N1,obs=cellinfo_N1)
adata_N1.var_names = geneinfo_N1.Gene
adata_N1.obs_names = cellinfo_N1.Cell
adata_N1


AnnData object with n_obs × n_vars = 184 × 13181 
    obs: 'Cell'
    var: 'Gene'

In [9]:
%%time

res_N1 = dca.autoencode(adata_N1, training_kwargs={'verbose': False})


### Autoencoder: Successfully preprocessed 13181 genes and 184 cells.
Calculating low dimensional representations...
Calculating reconstructions...
CPU times: user 8min 37s, sys: 4min 34s, total: 13min 11s
Wall time: 2min 29s


In [14]:
N1= res_N1.obsm['X_dca']
nr,nc = N1.shape
Br = ro.r.matrix(N1, nrow=nr, ncol=nc)
ro.r.assign("DCA_H1", Br)

3.275149,3.071579,3.348061,4.680077,...,84.591820,90.490662,119.731712,113.741104


In [ ]:
%%R
DCA_H1<-t(DCA_H1)
rownames(DCA_H1)<-geneinfo_N1[,1]
colnames(DCA_H1)<-cellinfo_N1[,1]

save(DCA_H1,file='/home/wenhao/RAW_REAL/DCA_norm/DCA_H1.RData')

In [13]:
%%R
dim(DCA_H1)

[1] 13181   184
